In [9]:
# importing required packages
import librosa
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [ ]:
# download the dataset with this link 
############# but the sample rate is decreased so less size , original is about 24GB
https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view

In [ ]:
# Ryerson Audio-Visual Database of Emotional Speech and Song dataset, 

In [ ]:
# Open the sound file with soundfile.SoundFile using with-as so it’s automatically closed once we’re done. 
#Read from it and call it X. Also, get the sample rate. If chroma is True, get the Short-Time Fourier Transform of X.

#Let result be an empty numpy array. Now, for each feature of the three, if it exists,
#make a call to the corresponding function from librosa.feature (eg- librosa.feature.mfcc for mfcc),
#and get the mean value. Call the function hstack() from numpy with result and the feature value, and store this in result. 
#hstack() stacks arrays in sequence horizontally (in a columnar fashion). Then, return the result.

In [2]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# 3. Now, let’s define a dictionary to hold numbers and the emotions available in the RAVDESS dataset,
#and a list to hold those we want- calm, happy, fearful, disgust.

In [3]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy','angry' 'fearful', 'disgust','surprised']

In [4]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("E:\\computer_vision\\audio\\speech emotion recognition\\speech_emotion_recognition_dataset\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [6]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [7]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
# Now, let’s initialize an MLPClassifier. This is a Multi-layer Perceptron Classifier;
# it optimizes the log-loss function using LBFGS or stochastic gradient descent. Unlike SVM or Naive Bayes,
# the MLPClassifier has an internal neural network for the purpose of classification. This is a feedforward ANN model.

In [36]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=400)

In [37]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [38]:
#Predict for the test set
y_pred=model.predict(x_test)

In [39]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 63.02%


In [ ]:
https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/